## 08_textgrid_manipulation
This notebook is going to be all about how to work with Textgrid files, like the ones we created in week 7 "forced alignment."

For the completely unitiated, a TextGrid is a filetype created by Praat that contains three types of information:

* time: start and stop of the transcript in seconds.
* text: the transcription
* tier: different tiers allow for multiple transcriptions at one time point. For example, you could have a "word" tier and a "phone" tier.

While you could theoretically load this as a textfile using default Python libraries, this is kinda clunky and we have some options for how to manipulate textgrids:

* [PraatIO](https://pypi.org/project/praatio/)
* [Parselmouth](https://parselmouth.readthedocs.io/en/stable/)
* [NLTK](https://www.nltk.org/) contains a textgrid module in the file `textgrid.py`, which is in a few different Hamilton Lab directories.

In [1]:
# I'll be using the NLTK textgrid module in this notebook, cause that's what I prefer personally.
from supplemental_files import textgrid

In [2]:
# Let's use the same textgrid we used in Week 2
# In fact this entire cell will be recap from week 2.
tg_fpath = './supplemental_files/OP0001_B1_spkr_02.textgrid'
with open(tg_fpath) as tgfile:
    tg = textgrid.TextGrid(tgfile.read()) # creates a special textgrid class using NLTK code
# Now we can view the tiers using tg.tiers
print(tg.tiers)
# Here's what the textgrid looks like
for row in tg.tiers[1].simple_transcript:
    print(row)

[<IntervalTier "phone" (0.01, 605.00) 100.00%>, <IntervalTier "word" (0.01, 605.00) 100.00%>]
('0.012471655328798186', '0.2619047619047619', 'sp')
('0.2619047619047619', '4.192970521541951', '{NS}')
('4.192970521541951', '5.160770975056689', 'MUM')
('5.160770975056689', '5.6197278911564625', 'STRONGLY')
('5.6197278911564625', '6.0587301587301585', 'DISLIKES')
('6.0587301587301585', '7.076417233560091', 'APPETIZERS')
('7.076417233560091', '7.794784580498866', 'sp')
('7.794784580498866', '11.167120181405894', '{NS}')
('11.167120181405894', '12.633786848072562', 'sp')
('12.633786848072562', '14.559410430839002', '{NS}')
('14.559410430839002', '14.958503401360543', 'THOSE')
('14.958503401360543', '15.068253968253966', 'WHO')
('15.068253968253966', '15.397505668934238', 'TEACH')
('15.397505668934238', '15.866439909297052', 'VALUES')
('15.866439909297052', '16.265532879818593', 'FIRST')
('16.265532879818593', '16.84421768707483', 'ABOLISH')
('16.84421768707483', '17.552607709750564', 'CHEATI

## Create phoneme matrices
A phoneme matrix is basically whether or not a certain phoneme is present at a specific timepoint (sample).

This section doesn't need much more introduction besides the fact that I will be loading in a list of all phonemes from a text file. This is in opposition to using `np.unique()` which would get us all the unique phonemes in the textgrid. The reason to use a text file is to keep the matrix layout the same across participants (just in case a particular phoneme is missing from one participant).

In [3]:
import csv
import numpy as np
import mne
from tqdm import tqdm

In [4]:
# Read in our phonemes, create a np array of the unique phoneme names
phoneme_list_fpath = "./supplemental_files/phonemes.txt"
phonemes = [] # init empty list
with open(phoneme_list_fpath, 'r') as my_csv:
    csvReader = csv.reader(my_csv)
    for row in csvReader:
        phonemes.append(row[0])
phonemes = np.array(phonemes)
print(phonemes.shape)

(76,)


In [5]:
# Read in textgrid
tg_fpath = './supplemental_files/OP0001_B1_spkr.textgrid'
with open(tg_fpath) as tgfile:
    tg = textgrid.TextGrid(tgfile.read())

In [6]:
# Read neural data
# This is the same neural data from Box that we used for notebooks 5/6.
# REMEMBER TO CHANGE THIS PATH IF YOU ARE FOLLOWING ALONG AT HOME
raw_fpath = "F:/Desktop/example_preprocessed_data.fif"
raw = mne.io.read_raw_fif(raw_fpath,preload=True)

Opening raw data file F:/Desktop/example_preprocessed_data.fif...
    Range : 0 ... 736914 =      0.000 ...  5756.772 secs
Ready.
Reading 0 ... 736914  =      0.000 ...  5756.772 secs...


<ipython-input-6-512b03f5c61c>:5: RuntimeWarning: This filename (F:/Desktop/example_preprocessed_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw = mne.io.read_raw_fif(raw_fpath,preload=True)


In [7]:
# These are probably slightly different due to a rounding error. Let's just ignore it lol
print(f"Raw is {raw.last_samp} samples long.")
print(f"Textgrid is {int(float(tg.tiers[1].simple_transcript[-1][1])*128)} samples long.")

Raw is 736914 samples long.
Textgrid is 736919 samples long.


In [8]:
# Create the phoenme matrix
# I am setting a limiter on this of 10,000 samples because this takes forever to run with the whole dataset.
# If you have a better way to write this code please let me know!
smin,smax = 50000,60000 # samples we will run 
phoneme_matrix = np.zeros((smax-smin,phonemes.shape[0]))
for row in tqdm(tg.tiers[1].simple_transcript):
    onset = int(float(row[0])*128)
    offset = int(float(row[1])*128)
    phn = row[2]
    for i in np.arange(smax-smin):
        if i+smin >= onset and i+smin <= offset:
            phn_idx = np.where(phonemes==phn)[0][0]
            phoneme_matrix[i][phn_idx] = 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12385/12385 [00:50<00:00, 244.19it/s]


## Create phonological feature matrices
A phonological feature matrix is the same thing as a phoneme feature matrix but with phonological features instead of phonemes. This is nice for mTRF, which we will learn about in the next notebook.

In [11]:
import re

In [12]:
features_dict = {
                'dorsal': ['y','w','k','kcl', 'g','gcl','eng','ng'],
                'coronal': ['ch','jh','sh','zh','s','z','t','tcl','d','dcl','n','th','dh','l','r'],
                'labial': ['f','v','p','pcl','b','bcl','m','em','w'],
                'high': ['uh','ux','uw','iy','ih','ix','ey','eh','oy'],
                'front': ['iy','ih','ix','ey','eh','ae','ay'],
                'low': ['aa','ao','ah','ax','ae','aw','ay','axr','ow','oy'],
                'back': ['aa','ao','ow','ah','ax','ax-h','uh','ux','uw','axr','aw'],
                'plosive': ['p','pcl','t','tcl','k','kcl','b','bcl','d','dcl','g','gcl','q'],
                'fricative': ['f','v','th','dh','s','sh','z','zh','hh','hv','ch','jh'],
                'syllabic': ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay','eh','ey','ih', 'ix', 'iy','ow', 'oy','uh', 'uw', 'ux'],
                'nasal': ['m','em','n','en','ng','eng','nx'],
                'voiced':   ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay','eh','ey','ih', 'ix', 'iy','ow', 'oy','uh', 'uw', 'ux','w','y','el','l','r','dh','z','v','b','bcl','d','dcl','g','gcl','m','em','n','en','eng','ng','nx','q','jh','zh'],
                'obstruent': ['b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx','f', 'g', 'gcl', 'hh', 'hv','jh', 'k', 'kcl', 'p', 'pcl', 'q', 's', 'sh','t', 'tcl', 'th','v','z', 'zh','q'],
                'sonorant': ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay','eh','ey','ih', 'ix', 'iy','ow', 'oy','uh', 'uw', 'ux','w','y','el','l','r','m', 'n', 'ng', 'eng', 'nx','en','em'],
        }
features = [f for f in features_dict.keys()]
print(features)
print(len(features))

['dorsal', 'coronal', 'labial', 'high', 'front', 'low', 'back', 'plosive', 'fricative', 'syllabic', 'nasal', 'voiced', 'obstruent', 'sonorant']
14


In [13]:
phnfeat_matrix = np.zeros((smax-smin,len(features)))
for row in tqdm(tg.tiers[1].simple_transcript):
    onset = int(float(row[0])*128)
    offset = int(float(row[1])*128)
    phn = row[2]
    for i in np.arange(smax-smin):
        if i+smin >= onset and i+smin <= offset:
            phn_stripped = re.sub(r'\d+', '', phn.lower())
            for fi, f in enumerate(features):
                if phn_stripped in features_dict[f]:
                    phnfeat_matrix[i,fi] = 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12385/12385 [00:50<00:00, 244.73it/s]


In [14]:
phnfeat_matrix

array([[0., 1., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Other random code

In [ ]:
# Fixing xmins: The Penn phonetics forced aligner (P2FA) sometimes doesn't set the
# start of the textgrids at exactly x=0. This can cause issues during concatenation.
# Here is some code I have used to fix that issue.
def fix_xmins(tg_fpath):
    '''
    Fix rounding error in textgrids.
    '''
    tab = "    "
    print("Fixing xmins for %s." % tg_fpath)
    fixed_textgrid = "%s_fixed.TextGrid" % tg_fpath
    with open(tg_fpath) as f:
        with open(fixed_textgrid, 'w') as newtext:
            for i, line in enumerate(f):
                if 'xm' in line: 
                    if tab+tab+tab in line:
                        indent = 19
                    elif tab+tab in line:
                        indent = 15
                    elif tab in line:
                        indent = 11
                    else:
                        indent = 7
                    time = float(line[indent:])
                    if 'xmin' in line:
                        if time < 1:
                            try:
                                newtext.write((line[:indent])+'0.0')
                                newtext.write('\n')
                            except:
                                raise Exception("Error at line %d. Transcription: %s" % (i, line))
                        else:
                            newtext.write(line)
                    if 'xmax' in line:
                        if split-time < 1:
                            newtext.write(line[:indent]+str(float(split)))
                            newtext.write('\n')
                        else:
                            newtext.write(line)
                else:
                    newtext.write(line)
    f.close()
    newtext.close()
    os.system("cp %s %s" % (fixed_textgrid, tg_fpath))
    os.system("rm %s" % fixed_textgrid)
    print("Done!")

In [ ]:
# Rounding textgrids
# written by Nicole Currens, a lab alumnus. Thanks, Nicole!
def is_number(s):
    '''
    checks whether a string is a number (float) or not
    written by Nicole Currens :nicoleparrot:
    '''
    try:
        float(s)
        return True
    except ValueError:
        return False
def round_textgrids(tg_fpath, short=False):
    '''
    Fixes rounding errors in the textgrids for a given subject/block/channel.
    Special thanks to Nicole Currens for helping me write this function!
    This currently ONLY works for 'ooTextFile' textgrids.
    TO DO: add support for 'ooTextFile short' textgrids!
    (This is currently broken and accessible via the bool `short`.)
    '''
    tab = "    "
    print("Rounding %s." % tg_fpath)
    rounded_textgrid = "%s_rounded.TextGrid" % tg_fpath
    with open(tg_fpath) as f:
        with open(rounded_textgrid, 'w') as newtext:
            # writes the header (i think?)
            i = 0 # this is janky, shut up. basically IDK how to enumerate w files?
            xms = [] # list of all xmin/xmax in our textgrid
            # if short == True:
            #    hdr_end = 12
            # else:
            #    hdr_end = 14
            for line in f:
                # print("Line %d" % i) 
                # rounds all numbers in the textgrid to 3 decimal points

                # for ooTextfile short textgrids - CURRENTLY DOESN'T WORK.
                # the forcedaligner makes these instead of regular-ass
                # textgrids ... so it would be nice if we could round these
                # directly. god i wish life was easy sometimes
                if short == True:
                    if is_number(line):
                        time = float(line)
                        newnum = round(float(time-0.0005), 3) # janky
                        xms.append(newnum)

                        # this fixes a rounding error and idk how really
                        # I'M BAD AT CODING OK
                        if xmin == True:
                            if len(xms) > 5:
                                if xms[i] != xms[i-3]:
                                    if xms[i] != None and xms[i-3] != None:
                                        if abs(xms[i]-xms[i-3]) < 0.1:
                                            newnum = xms[i-3]
                            xmin = False
                        else:
                            xmin = True
                        newtext.write(str(newnum))
                        newtext.write('\n')
                    else:
                        newtext.write(line)
                        xms.append(None)
                # for ooTextfile textgrids
                else:
                    if 'xm' in line:
                        # get our spacing
                        # this is crap code but IDK how to do this.
                        if tab+tab+tab in line:
                            indent = 19
                        elif tab+tab in line:
                            indent = 15
                        elif tab in line:
                            indent = 11
                        else:
                            indent = 7
                        # print(line)
                        # if i >= hdr_end: # if not the header
                        # 	indent = 19
                        # else: # if the header
                        #  	indent = 7
                        #  	print(i)
                        #  	print("This is the header line:")
                        #  	print(line[indent:])
                        #  	print("^^^")
                        #  	print("This is the full line:")
                        #  	print(line)
                        time = float(line[indent:])
                        newnum = round(float(time-0.0005), 3) # janky
                        xms.append(newnum)
                        # this fixes a rounding error and idk how really
                        # I'M BAD AT CODING OK
                        if 'xmin' in line:
                            if len(xms) > 5:
                                if xms[i] != xms[i-3]:
                                    if xms[i] != None and xms[i-3] != None:
                                        if abs(xms[i]-xms[i-3]) < 0.1:
                                            newnum = xms[i-3]
                        newtext.write(line[:indent] + str(newnum))
                        newtext.write('\n')

                    else:
                        newtext.write(line)
                        xms.append(None)
                i += 1
    f.close()
    newtext.close()
    # overwrite original textgrid and delete rounded file
    os.system("cp %s %s" % (rounded_textgrid, tg_fpath))
    os.system("rm %s" % rounded_textgrid)
    print("Done!")